# ComfyUI FLUX Setup for Google Colab
This notebook adapts the `flux_install.sh` script for use in Google Colab to set up ComfyUI with FLUX-specific nodes and dependencies.

## 1. Configuration

## 1.1. Mount Google Drive (Optional)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# You can then access your Drive files at /content/drive/MyDrive/

In [ ]:
# --- Configuration ---
COMFYUI_DIR = "/content/drive/MyDrive/ComfyUI"  # Main directory for ComfyUI installation in Colab
UPDATE_COMFY_UI = True       # Set to False to disable ComfyUI git pull if already cloned
INSTALL_CUSTOM_NODES_DEPENDENCIES = True
USE_COMFYUI_MANAGER = True
INSTALL_SAGEATTENTION = "background" # "true", "false", or "background" (background not directly applicable in Colab, will be 'true')

print(f"Target Directory: {COMFYUI_DIR}")
print(f"Update ComfyUI: {UPDATE_COMFY_UI}")
print(f"Install Custom Nodes & Dependencies: {INSTALL_CUSTOM_NODES_DEPENDENCIES}")
print(f"Use ComfyUI Manager: {USE_COMFYUI_MANAGER}")
print(f"SageAttention Installation: {INSTALL_SAGEATTENTION}")

## 2. Create Directories and Clone ComfyUI

In [ ]:
import os

# Create parent directory for COMFYUI_DIR if it doesn't exist
if not os.path.exists(os.path.dirname(COMFYUI_DIR)):
    os.makedirs(os.path.dirname(COMFYUI_DIR))

os.chdir(os.path.dirname(COMFYUI_DIR)) # Navigate to parent directory

if not os.path.exists(COMFYUI_DIR):
    print("--- Cloning ComfyUI ---")
    !git clone https://github.com/comfyanonymous/ComfyUI.git "{COMFYUI_DIR}"
    os.chdir(COMFYUI_DIR)
else:
    os.chdir(COMFYUI_DIR)
    if UPDATE_COMFY_UI:
        print("--- Updating ComfyUI ---")
        !git pull
    else:
        print("--- ComfyUI exists, skipping update as per configuration. ---")

## 3. Install ComfyUI Main Dependencies

In [ ]:
print("--- Installing ComfyUI main dependencies (from requirements.txt) ---")
%pip install -r requirements.txt

## 4. Install Additional Core Dependencies

In [ ]:
print("--- Installing additional core dependencies ---")
%pip install accelerate einops transformers>=4.28.1 safetensors>=0.4.2 aiohttp pyyaml Pillow scipy tqdm psutil tokenizers>=0.13.3
%pip install torchsde
%pip install kornia>=0.7.1 spandrel soundfile sentencepiece
%pip install imageio-ffmpeg  # For VHS_VideoCombine node

## 5. Install prerequisites for ComfyUI-Manager CLI

In [ ]:
print("--- Installing GitPython and Typer (for ComfyUI-Manager CLI) ---")
%pip install GitPython typer

## 6. Setup ComfyUI-Manager

In [ ]:
if USE_COMFYUI_MANAGER:
    print("--- Setting up ComfyUI-Manager ---")
    MANAGER_DIR = os.path.join(COMFYUI_DIR, "custom_nodes/ComfyUI-Manager")
    if not os.path.exists(MANAGER_DIR):
        print("Cloning ComfyUI-Manager...")
        !git clone https://github.com/ltdrdata/ComfyUI-Manager.git "{MANAGER_DIR}"
    else:
        print("Updating ComfyUI-Manager...")
        os.chdir(MANAGER_DIR)
        !git pull
        os.chdir(COMFYUI_DIR) # Return to ComfyUI base

    # Install requirements for ComfyUI-Manager
    manager_req_file = os.path.join(MANAGER_DIR, "requirements.txt")
    if os.path.exists(manager_req_file):
        print("Installing ComfyUI-Manager's own requirements...")
        %pip install -r "{manager_req_file}"

    print("Attempting to restore dependencies via ComfyUI-Manager CLI...")
    # Note: The cm-cli.py script might need adjustments or specific handling in Colab
    # For now, we call it directly. Ensure it's executable if needed.
    !python "{os.path.join(MANAGER_DIR, 'cm-cli.py')}" restore-dependencies

## 7. Install Custom Nodes and Their Dependencies

In [ ]:
if INSTALL_CUSTOM_NODES_DEPENDENCIES:
    print("--- Installing Custom Nodes & Their Dependencies ---")
    CUSTOM_NODES_DIR = os.path.join(COMFYUI_DIR, "custom_nodes")
    if not os.path.exists(CUSTOM_NODES_DIR):
        os.makedirs(CUSTOM_NODES_DIR)
    os.chdir(CUSTOM_NODES_DIR)

    CUSTOM_NODES_REPOS = [
        "https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite",
        "https://github.com/civitai/civitai_comfy_nodes",
        "https://github.com/kijai/ComfyUI-KJNodes",
        "https://github.com/cubiq/ComfyUI_essentials",
        "https://github.com/sipherxyz/comfyui-art-venture",
        "https://github.com/twri/sdxl_prompt_styler",
        "https://github.com/Nourepide/ComfyUI-Allor",
        "https://github.com/Extraltodeus/sigmas_tools_and_the_golden_scheduler",
        "https://github.com/rgthree/rgthree-comfy",
        "https://github.com/pollockjj/ComfyUI-MultiGPU",
        "https://github.com/daxcay/ComfyUI-JDCN",
        "https://github.com/city96/ComfyUI-GGUF",
        "https://github.com/calcuis/gguf",
        "https://github.com/kijai/ComfyUI-GIMM-VFI",
        "https://github.com/kijai/ComfyUI-DepthAnythingV2",
        "https://github.com/kijai/ComfyUI-WanVideoWrapper.git",
    ]

    for repo_url in CUSTOM_NODES_REPOS:
        repo_name = repo_url.split('/')[-1].replace('.git', '')
        if not os.path.exists(repo_name):
            print(f"Cloning {repo_name}...")
            !git clone {repo_url} {repo_name}
        else:
            print(f"Updating {repo_name}...")
            os.chdir(repo_name)
            !git pull
            os.chdir(CUSTOM_NODES_DIR) # Go back to custom_nodes directory

    # Install requirements for specific nodes
    NODES_WITH_REQS = ["ComfyUI-GGUF", "ComfyUI-JDCN", "ComfyUI-KJNodes,ComfyUI-WanVideoWrapper"]
    for node_name in NODES_WITH_REQS:
        node_req_file = os.path.join(CUSTOM_NODES_DIR, node_name, "requirements.txt")
        if os.path.exists(node_req_file):
            print(f"Installing dependencies for {node_name}...")
            %pip install -r "{node_req_file}"

    # Special handling for was-node-suite-comfyui
    WAS_NODE_DIR_NAME = "was-node-suite-comfyui"
    WAS_NODE_REPO = "https://github.com/WASasquatch/was-node-suite-comfyui.git"
    if not os.path.exists(WAS_NODE_DIR_NAME):
        print(f"Cloning {WAS_NODE_DIR_NAME}...")
        !git clone {WAS_NODE_REPO} {WAS_NODE_DIR_NAME}
    else:
        print(f"Updating {WAS_NODE_DIR_NAME}...")
        os.chdir(WAS_NODE_DIR_NAME)
        !git pull
        os.chdir(CUSTOM_NODES_DIR) # Go back to custom_nodes directory
    
    was_req_file = os.path.join(CUSTOM_NODES_DIR, WAS_NODE_DIR_NAME, "requirements.txt")
    if os.path.exists(was_req_file):
        print(f"Installing dependencies for {WAS_NODE_DIR_NAME}...")
        %pip install -r "{was_req_file}"

    os.chdir(COMFYUI_DIR) # Return to the main ComfyUI directory

## 8. Clone FFmpeg (Optional - for building from source, usually not needed in Colab)

In [ ]:
print("--- Cloning FFmpeg source (optional) ---")
os.chdir(COMFYUI_DIR)
if not os.path.exists("ffmpeg"):
    print("Cloning FFmpeg repository...")
    !git clone https://git.ffmpeg.org/ffmpeg.git ffmpeg # Clone into a directory named ffmpeg
else:
    print("FFmpeg directory already exists. Skipping clone.")

## 9. Install gguf-node

In [ ]:
print("--- Installing gguf-node ---")
%pip install gguf-node

## 10. Install SageAttention

In [ ]:
# In Colab, 'background' installation isn't directly applicable like in a shell script.
# We'll treat 'background' as 'true'.
if INSTALL_SAGEATTENTION != "false":
    print(f"--- Setting up SageAttention (mode: {INSTALL_SAGEATTENTION}) ---")
    os.chdir(COMFYUI_DIR) # Ensure we are in the ComfyUI directory
    SAGE_DIR = os.path.join(COMFYUI_DIR, "SageAttention")
    if not os.path.exists(SAGE_DIR):
        print("Cloning SageAttention repository...")
        !git clone https://github.com/thu-ml/SageAttention.git SageAttention
    else:
        print("SageAttention directory already exists. Updating...")
        os.chdir(SAGE_DIR)
        !git pull
        os.chdir(COMFYUI_DIR) # Return to ComfyUI base

    # Install SageAttention
    # The original script had a complex backgrounding logic here.
    # For Colab, we'll do a direct install.
    print("Installing SageAttention... (This may take a while)")
    os.chdir(SAGE_DIR) # Navigate into SageAttention directory
    !python setup.py install
    os.chdir(COMFYUI_DIR) # Return to ComfyUI base
    print("SageAttention installation attempted.")
else:
    print("--- Skipping SageAttention installation as per configuration. ---")

## 11. Download Models

This section handles the acquisition of models.
First, it ensures that `model_configs.json` (which defines the available models and their download links) is fetched from the `bbits10/comfy_template` GitHub repository and placed into your ComfyUI directory.
Following that, an interactive interface will be set up, allowing you to select and download specific models from the `model_configs.json` directly within this notebook.

In [ ]:
# This section corresponds to model_downloader.py
print("--- Model Configuration Setup ---")

# 1. Define your GitHub repository and the path to the files
YOUR_REPO_URL = "https://github.com/bbits10/comfy_template.git"
PATH_TO_FILES_IN_REPO = "" # model_configs.json is in the root

# 2. Clone your repository (or ensure it's cloned)
print(f"Cloning repository: {YOUR_REPO_URL} to fetch model_configs.json")
REPO_NAME = YOUR_REPO_URL.split('/')[-1].replace('.git', '')
CLONED_REPO_PATH = f"/content/{REPO_NAME}"

if not os.path.exists(CLONED_REPO_PATH):
    !git clone {YOUR_REPO_URL} {CLONED_REPO_PATH}
else:
    print(f"Repository already cloned at {CLONED_REPO_PATH}. Ensuring it's up to date.")
    !cd {CLONED_REPO_PATH} && git pull

# 3. Copy model_configs.json to COMFYUI_DIR
print(f"Copying model_configs.json to {COMFYUI_DIR}")
MODEL_CONFIGS_JSON_SOURCE = os.path.join(CLONED_REPO_PATH, PATH_TO_FILES_IN_REPO, "model_configs.json")
MODEL_CONFIGS_JSON_DEST = os.path.join(COMFYUI_DIR, "model_configs.json")

if os.path.exists(MODEL_CONFIGS_JSON_SOURCE):
    !cp -f {MODEL_CONFIGS_JSON_SOURCE} {MODEL_CONFIGS_JSON_DEST}
    print(f"Copied model_configs.json to {MODEL_CONFIGS_JSON_DEST}")
else:
    print(f"ERROR: model_configs.json not found at {MODEL_CONFIGS_JSON_SOURCE}. Model selection UI may not work correctly.")

# --- Interactive Model Downloader Cell ---
print("\n--- Interactive Model Downloader --- ")

import json
import os
import subprocess
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Configuration for the interactive downloader part
CONFIG_FILE_PATH_FOR_INTERACTIVE_DOWNLOADER = MODEL_CONFIGS_JSON_DEST # Use the path where we copied it
# COMFYUI_BASE_DIR is already defined earlier in the notebook
MODELS_DESTINATION_ROOT_INTERACTIVE = os.path.join(COMFYUI_DIR, 'models')

if not os.path.exists(MODELS_DESTINATION_ROOT_INTERACTIVE):
    os.makedirs(MODELS_DESTINATION_ROOT_INTERACTIVE, exist_ok=True)

# Helper Functions for Interactive Downloader
def load_model_configs_interactive(file_path):
    if not os.path.exists(file_path):
        print(f"Error: Configuration file '{file_path}' not found for interactive downloader.")
        return {}
    try:
        with open(file_path, 'r') as f:
            return json.load(f)
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from '{file_path}'.")
        return {}

model_configs_interactive = load_model_configs_interactive(CONFIG_FILE_PATH_FOR_INTERACTIVE_DOWNLOADER)

# UI Widget Definitions
if model_configs_interactive: # Only create widgets if config loaded
    model_set_dropdown_interactive = widgets.Dropdown(
        description='Model Set:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='50%')
    )
    models_checkboxes_container_interactive = widgets.VBox()
    download_button_interactive = widgets.Button(
        description='Download Selected Models',
        button_style='success',
        tooltip='Click to download checked models',
        icon='download'
    )
    output_area_interactive = widgets.Output()
else:
    display(HTML("<p style='color:red; font-weight:bold;'>Interactive model downloader UI cannot be initialized: model_configs.json was not loaded.</p>"))

# Callback Functions for Interactive Downloader
def update_model_set_options_interactive():
    if not model_configs_interactive:
        model_set_dropdown_interactive.options = [("No config loaded", None)]
        return
    options = [(model_configs_interactive[key].get("name", key), key) for key in model_configs_interactive.keys()]
    model_set_dropdown_interactive.options = [("Select a model set...", None)] + options

def on_model_set_change_interactive(change):
    with output_area_interactive:
        clear_output()
    selected_set_key = change.get('new')
    new_checkboxes = []
    if selected_set_key and selected_set_key in model_configs_interactive:
        current_model_group = model_configs_interactive[selected_set_key]
        group_display_name = current_model_group.get("name", selected_set_key)
        current_models = current_model_group.get("models", {})
        if not current_models:
            new_checkboxes.append(widgets.HTML(f"<em>No models found in '{group_display_name}'.</em>"))
        else:
            for model_key, model_info in current_models.items():
                cb = widgets.Checkbox(description=model_info.get("name", model_key), value=False, indent=False, layout=widgets.Layout(width='95%'))
                cb.model_key_ref = model_key
                cb.set_key_ref = selected_set_key
                new_checkboxes.append(cb)
    models_checkboxes_container_interactive.children = tuple(new_checkboxes)

def download_individual_model_interactive(model_data, comfyui_root_dir_for_dl, local_output_area):
    model_name = model_data.get('name', 'Unknown Model')
    model_url = model_data.get('url')
    model_relative_path = model_data.get('path')
    if not model_url or not model_relative_path:
        with local_output_area:
            print(f"Skipping {model_name}: Missing URL or path in config.")
        return
    model_save_path = os.path.join(comfyui_root_dir_for_dl, 'models', model_relative_path)
    try:
        os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
    except Exception as e:
        with local_output_area:
            print(f"Error creating directory for {model_name} at {os.path.dirname(model_save_path)}: {e}")
        return
    with local_output_area:
        print(f"Starting download for: {model_name}")
        print(f"  URL: {model_url}")
        print(f"  Saving to: {model_save_path}")
        try:
            command = ['wget', '-O', model_save_path, model_url] # Using wget for Colab
            process = subprocess.run(command, capture_output=True, text=True, check=False)
            if process.returncode == 0:
                print(f"Successfully downloaded: {model_name}")
            else:
                print(f"Error downloading {model_name} (Code: {process.returncode}):")
                if process.stdout: print(f"  Stdout: {process.stdout.strip()}")
                if process.stderr: print(f"  Stderr: {process.stderr.strip()}")
        except FileNotFoundError:
            print(f"Error: '{command[0]}' command not found. Please ensure it is installed.")
        except Exception as e:
            print(f"An unexpected error occurred while downloading {model_name}: {e}")
        print("-" * 30)

def on_download_button_clicked_interactive(b):
    with output_area_interactive:
        clear_output(wait=True)
        print("Preparing to download selected models...")
    selected_models_to_download_data = []
    for cb in models_checkboxes_container_interactive.children:
        if isinstance(cb, widgets.Checkbox) and cb.value:
            set_key = cb.set_key_ref
            model_key = cb.model_key_ref
            if set_key in model_configs_interactive and model_key in model_configs_interactive[set_key]["models"]:
                model_data = model_configs_interactive[set_key]["models"][model_key]
                selected_models_to_download_data.append(model_data)
    if not selected_models_to_download_data:
        with output_area_interactive:
            print("No models selected for download.")
        return
    with output_area_interactive:
        print(f"Found {len(selected_models_to_download_data)} model(s) to download to base: {MODELS_DESTINATION_ROOT_INTERACTIVE}")
        print("-" * 30)
    for model_data_to_dl in selected_models_to_download_data:
        download_individual_model_interactive(model_data_to_dl, COMFYUI_DIR, output_area_interactive) # Pass COMFYUI_DIR as the root for model saving structure
    with output_area_interactive:
        print("All selected downloads attempted.")

# Initialize UI and Display for Interactive Downloader
if model_configs_interactive: # Only display if config loaded
    model_set_dropdown_interactive.observe(on_model_set_change_interactive, names='value')
    download_button_interactive.on_click(on_download_button_clicked_interactive)
    update_model_set_options_interactive()
    display(widgets.VBox([
        widgets.HTML("<h3>Select Models to Download:</h3>"),
        model_set_dropdown_interactive, 
        models_checkboxes_container_interactive, 
        download_button_interactive, 
        output_area_interactive
    ]))
else:
    print("Skipping display of interactive model downloader UI as model_configs.json was not loaded.")

## 12. Final Steps & Starting ComfyUI

In [ ]:
print("--- Setup Complete ---")
print(f"ComfyUI should be installed in: {COMFYUI_DIR}")
print("Starting ComfyUI with cloudflared tunnel...")

# 1. Download and Install cloudflared
print("\n--- Installing cloudflared ---")
!wget -q -P /tmp https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i /tmp/cloudflared-linux-amd64.deb

# 2. Define function to start cloudflared tunnel
import subprocess
import threading
import time
import socket
import urllib.request
import os

comfyui_port = 8188 # Default ComfyUI port

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared...")
  print("(If it gets stuck here, cloudflared might be having issues or ComfyUI didn't start correctly.)\n")

  # Start cloudflared tunnel
  p = subprocess.Popen(["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com" in l:
      print(f"\n!!! THIS IS YOUR PUBLIC URL TO ACCESS COMFYUI !!!\n{l[l.find('https'):].strip()}\n")
    # You can uncomment the next line to see all cloudflared output for debugging
    # print(l, end='')

# 3. Start the ComfyUI server and the cloudflared tunnel
os.chdir(COMFYUI_DIR) # Navigate to ComfyUI directory

# Start cloudflared in a background thread
threading.Thread(target=iframe_thread, daemon=True, args=(comfyui_port,)).start()

print(f"\n--- Starting ComfyUI on port {comfyui_port} ---")
# Start ComfyUI. --dont-print-server is used as cloudflared provides the URL.
# Use --preview-method auto for better preview generation in Colab if needed.
!python main.py --listen --port {comfyui_port} --dont-print-server --preview-method auto